<a href="https://colab.research.google.com/github/jpmachinelearning/U4_S16_Trabajo_Final_2P_PLAZA_A-PADILLA_J/blob/main/src/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyGithub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.7/432.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.6 MB/s eta 0:00:00


In [2]:
# Obteniendo datos
!pip install PyGithub
from google.colab import files, userdata
from github import Github, Auth, GithubException
import pandas as pd
import os

# ==========================================
# 1. CONFIGURACIÓN
# ==========================================
try:
    TOKEN = userdata.get('GITHUB_TOKEN')
    # Configuración para Git por comandos
    USER_GITHUB = "jpmachinelearning" # Tu usuario de GitHub
    EMAIL_GITHUB = "tu_email@ejemplo.com" # Cambia esto por tu email de GitHub
except userdata.SecretNotFoundError:
    print("❌ Error: Asegúrate de crear el secreto 'GITHUB_TOKEN' en el panel de la izquierda.")

# Repositorio fuente para extracción
REPO_NAME = 'OCA/l10n-spain'

# Repositorio destino para guardar la data (formato para Git)
DEST_REPO_NAME = 'U4_S16_Trabajo_Final_2P_PLAZA_A-PADILLA_J'
REPO_URL = f"https://{TOKEN}@github.com/{USER_GITHUB}/{DEST_REPO_NAME}.git"

# Configuración de archivos
nombre_archivo = 'datos_github.csv'
ruta_local = os.path.join('/content/sample_data', nombre_archivo)

# Límite de registros
LIMITE_REGISTROS = 200

# ==========================================
# 2. CONFIGURACIÓN DE GIT (PULL)
# ==========================================
# Clonamos el repo si no existe, o hacemos pull si ya está
%cd /content/
if not os.path.exists(DEST_REPO_NAME):
    print(f"🚀 Clonando repositorio {DEST_REPO_NAME}...")
    !git clone {REPO_URL}
else:
    print(f"🔄 Sincronizando repositorio (git pull)...")
    %cd {DEST_REPO_NAME}
    !git pull origin main
    %cd /content/

# Configurar identidad de Git
!git config --global user.email "{EMAIL_GITHUB}"
!git config --global user.name "{USER_GITHUB}"

# ==========================================
# 2. CONEXIÓN API (Extracción)
# ==========================================
try:
    auth = Auth.Token(TOKEN)
    g = Github(auth=auth)

    repo = g.get_repo(REPO_NAME)
    print(f"✅ Conectado exitosamente al proyecto: {repo.full_name}")

    data = []

    # ==========================================
    # 3. EXTRACCIÓN DE ISSUES (RF-01)
    # ==========================================
    print(f"\n--- Extrayendo hasta {LIMITE_REGISTROS} Issues ---")
    issues = repo.get_issues(state='all', sort='created', direction='desc')
    count = 0

    for item in issues:
        if count >= LIMITE_REGISTROS: break
        if item.pull_request is not None: continue

        print(f"📥 Procesando Issue #{item.number}...")
        comments = [c.body for c in item.get_comments() if c.body]

        data.append({
            'tipo': 'Issue',
            'numero': item.number,
            'comentarios': " [SEP] ".join(comments)
        })
        count += 1

    # ==========================================
    # 4. EXTRACCIÓN DE PULL REQUESTS (RF-01)
    # ==========================================
    print(f"\n--- Extrayendo hasta {LIMITE_REGISTROS} Pull Requests ---")
    pulls = repo.get_pulls(state='all', sort='created', direction='desc')
    count = 0

    for pr in pulls:
        if count >= LIMITE_REGISTROS: break

        print(f"📥 Procesando PR #{pr.number}...")
        comments = [c.body for c in pr.get_issue_comments() if c.body]

        data.append({
            'tipo': 'Pull Request',
            'numero': pr.number,
            'comentarios': " [SEP] ".join(comments)
        })
        count += 1

    # ==========================================
    # 5. GUARDAR RESULTADOS LOCALMENTE
    # ==========================================
    if not data:
        print("⚠️ No se encontraron datos.")
    else:
        df = pd.DataFrame(data)

        # Guardar en /content/sample_data/datos_github.csv
        df.to_csv(ruta_local, index=False, encoding='utf-8-sig')

        print("-" * 30)
        print(f"✨ RESULTADOS DE LA RECOLECCIÓN:")
        print(f"📁 Guardado local en: {ruta_local}")
        print(f"📊 Total de filas (Issues + PRs): {len(df)}")
        print("-" * 30)

        # ==========================================
        # 6. ENVIAR A REPOSITORIO USANDO GIT PUSH
        # ==========================================
        print(f"\n📦 Moviendo archivo al repositorio y subiendo cambios...")

        # Asegurar que la carpeta data existe en el repo clonado
        repo_path = f"/content/{DEST_REPO_NAME}/data"
        os.makedirs(repo_path, exist_ok=True)

        # Copiar de sample_data al repositorio
        !cp {ruta_local} {repo_path}/{nombre_archivo}

        # Comandos de Git para subir
        %cd /content/{DEST_REPO_NAME}
        !git add data/{nombre_archivo}
        !git commit -m "Actualización de datos_github.csv via Git Pull/Push desde Colab"
        !git push origin main

        print(f"\n✅ Proceso completado. Archivo disponible en GitHub y localmente en sample_data.")
        %cd /content/

except Exception as e:
    print(f"❌ Error: {e}")

# ==========================================
# 6. ENVIAR A REPOSITORIO (CON MANEJO DE CONFLICTOS)
# ==========================================
print(f"\n🚀 Sincronizando y subiendo cambios a GitHub...")

# Ruta del repo y carpeta data
repo_path = f"/content/{DEST_REPO_NAME}"
data_folder = os.path.join(repo_path, "data")
os.makedirs(data_folder, exist_ok=True)

# Copiar el archivo fresco de sample_data al repo clonado
!cp {ruta_local} {data_folder}/{nombre_archivo}

%cd {repo_path}

# 1. Preparar el commit local
!git add data/{nombre_archivo}
# Usamos un commit con fecha para diferenciar
import datetime
timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
!git commit -m "Auto-update data: {timestamp}"

# 2. Intentar traer cambios remotos con Rebase
# La estrategia '-X ours' le dice a Git: "Si hay conflicto en una línea,
# mantén lo que acabo de generar en este Colab".
print("🔄 Revisando si hay cambios externos (Pull Rebase)...")
status = os.system("git pull --rebase -X ours origin main")

if status == 0:
    # 3. Empujar los cambios
    print("📤 Subiendo cambios finales...")
    push_status = os.system("git push origin main")
    if push_status == 0:
        print(f"✅ ¡Éxito! Datos sincronizados en el repositorio.")
    else:
        print("❌ Error al hacer push. Revisa permisos del Token.")
else:
    print("⚠️ Hubo un problema al sincronizar. Intentando limpiar estado de Git...")
    !git rebase --abort

%cd /content/

/content
🚀 Clonando repositorio U4_S16_Trabajo_Final_2P_PLAZA_A-PADILLA_J...
Cloning into 'U4_S16_Trabajo_Final_2P_PLAZA_A-PADILLA_J'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 23 (delta 6), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 122.51 KiB | 13.61 MiB/s, done.
Resolving deltas: 100% (6/6), done.
✅ Conectado exitosamente al proyecto: OCA/l10n-spain

--- Extrayendo hasta 200 Issues ---
📥 Procesando Issue #4793...
📥 Procesando Issue #4760...
📥 Procesando Issue #4743...
📥 Procesando Issue #4681...
📥 Procesando Issue #4677...
📥 Procesando Issue #4666...
📥 Procesando Issue #4652...
📥 Procesando Issue #4650...
📥 Procesando Issue #4649...
📥 Procesando Issue #4642...
📥 Procesando Issue #4641...
📥 Procesando Issue #4639...
📥 Procesando Issue #4637...
📥 Procesando Issue #4632...
📥 Procesando Issue #4631...
📥 Procesando Issue #4626...
📥 Procesando Issue 

In [3]:
file_path = '/content/U4_S16_Trabajo_Final_2P_PLAZA_A-PADILLA_J/data/datos_github.csv'
#data = pd.read_csv(file_path, sep='\s+', header=None)
data = pd.read_csv(file_path)
data

,tipo,numero,comentarios
0,Issue,4793,Siendo tratado en #4784
1,Issue,4760,"Además, hemos podido comprobar que el mismo pr..."
2,Issue,4743,NaN
3,Issue,4681,Hola.\n\nAgradecería si alguien puede confirma...
4,Issue,4677,NaN
...,...,...,...
395,Pull Request,4569,"Hey, thanks for contributing! Proceeding to me..."
396,Pull Request,4568,"Hi @pedrobaeza,\nsome modules you are maintain..."
397,Pull Request,4566,"Hi @EmilioPascual, @rafaelbn,\nsome modules yo..."
398,Pull Request,4565,/ocabot merge major [SEP] This PR looks fantas...


In [4]:
data.count()

,0
tipo,400
numero,400
comentarios,356


In [5]:
data.shape

(400, 3)

In [6]:
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# --- Descarga de recursos necesarios (incluyendo los que sugeriste) ---
# Intentamos descargar todo lo necesario; si ya existe, nltk lo omite.
try:
    nltk.download('stopwords')
    nltk.download('punkt')
    nltk.download('punkt_tab') # Recurso sugerido
    nltk.download('averaged_perceptron_tagger') # Necesario para pos_tag estándar
    nltk.download('averaged_perceptron_tagger_eng') # Recurso sugerido explícitamente
except Exception as e:
    print(f"Nota sobre descargas NLTK: {e}")

# --- 1. Función de EDA (Filtrado de documentos) ---
def eda(df):
    """
    Lee el dataframe, selecciona la columna 'comentarios', elimina vacíos,
    filtra mensajes de bots (ocabot) y textos que no sean mayoritariamente español.
    """
    # Eliminar filas donde 'comentarios' es NaN (vacío)
    df_limpio = df.dropna(subset=['comentarios']).copy()

    documentos_validos = []

    # Pre-cargamos stopwords para la heurística de idioma
    stop_es = set(stopwords.words('spanish'))
    stop_en = set(stopwords.words('english'))

    for texto in df_limpio['comentarios']:
        # Asegurar que es string
        if not isinstance(texto, str):
            continue

        texto_lower = texto.lower()

        # --- A. Filtro de Bots ---
        # Si contiene 'ocabot' o comandos típicos de bot, lo saltamos
        if 'ocabot' in texto_lower or 'merge command' in texto_lower:
            continue

        # Limpieza básica de separadores de hilos para análisis
        # Reemplazamos [SEP] y saltos de línea por espacios
        texto_analisis = texto.replace('[SEP]', ' ').replace('\n', ' ')

        # --- B. Filtro de Idioma / Código ---
        # Usamos una heurística simple: Contar stopwords
        tokens = word_tokenize(texto_analisis)

        # Contamos cuántas palabras funcionales del español y del inglés aparecen
        count_es = sum(1 for w in tokens if w.lower() in stop_es)
        count_en = sum(1 for w in tokens if w.lower() in stop_en)

        # REGLA: Para ser aceptado, debe tener más "señales" de ser español que de inglés/código
        # y al menos 1 palabra funcional en español (para evitar bloques puros de código)
        if count_es > count_en and count_es > 0:
            documentos_validos.append(texto_analisis)

    return documentos_validos

# --- 2. Función de Preprocesado (Base modificada mínimamente) ---
def preprocesado(documento):
    documento = documento.lower()
    tokens_nltk = nltk.word_tokenize(documento)

    # Nota: pos_tag usa modelos en inglés por defecto, pero sirve para separar puntuación
    tags = nltk.pos_tag(tokens_nltk)

    puntuacion = ['--', '_', ':', ')', "'", '"--', ',', ']', '!', '"', '?', '-', '--------', '[', ';', '.', '(', '----------', "'--", '&', "''", '``']

    # Corrección menor: asegurar que filtramos si el TAG o la PALABRA es puntuación
    # (El código original solo miraba 'tag', pero a veces nltk etiqueta raro en español)
    tokens_nltk_limpio = [word for word, tag in tags if tag not in puntuacion and word not in puntuacion]

    tokens_nltk_limpio = [t.strip('_') for t in tokens_nltk_limpio]

    # Corrección lógica del original: el original tenía "if '[' remove ']'"
    if '[' in tokens_nltk_limpio: tokens_nltk_limpio.remove('[')
    if ']' in tokens_nltk_limpio: tokens_nltk_limpio.remove(']') # Corregido para remover el corchete de cierre

    stop_words = set(stopwords.words('spanish'))
    # stop_words.remove('no')
    # stop_words.update('así')

    # Filtro final de stopwords y alfanuméricos
    tokens_nltk_limpio = [w for w in tokens_nltk_limpio if w not in stop_words and w.isalnum()]

    del tokens_nltk, tags, puntuacion
    return tokens_nltk_limpio

# --- 3. Flujo de Trabajo Principal ---

# Cargar dataset
try:
    #file_path = '/content/U4_S16_Trabajo_Final_2P_PLAZA_A-PADILLA_J/data/datos_github.csv'
    df = pd.read_csv(file_path)
    print("Dataset cargado exitosamente.")

    # Paso 1: EDA
    # Obtenemos la lista de textos "crudos" que pasaron el filtro de calidad
    textos_filtrados = eda(df)
    print(f"Documentos válidos tras filtro EDA: {len(textos_filtrados)}")

    # Paso 2: Generar Colección
    # Aplicamos preprocesado a cada texto válido
    colecciones = [preprocesado(doc) for doc in textos_filtrados]

    # Limpieza final: eliminar listas vacías que pudieron quedar (ej. mensajes que eran solo stopwords)
    colecciones = [c for c in colecciones if len(c) > 0]

    print(f"Colección final generada con {len(colecciones)} documentos preprocesados.")

    # Muestra del resultado
    print("\nEjemplo de los primeros 2 documentos procesados:")
    for i, doc in enumerate(colecciones[:2]):
        print(f"Doc {i+1}: {doc}")

except FileNotFoundError:
    print("Error: No se encontró el archivo 'datos_github_proyecto_final.csv'")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


Dataset cargado exitosamente.
Documentos válidos tras filtro EDA: 191
Colección final generada con 191 documentos preprocesados.

Ejemplo de los primeros 2 documentos procesados:
Doc 1: ['siendo', 'tratado', '4784']
Doc 2: ['además', 'podido', 'comprobar', 'mismo', 'problema', 'reproduce', 'odoo', 'utilizando', 'módulo', 'rama', 'generar', 'fichero', 'modelo', '190', 'odoo', 'ejercicio', '2025', 'sede', 'electrónica', 'aeat', 'devuelve', 'errores', 'similares', 'relacionados', 'falta', 'campos', 'obligatorios', 'correspondientes', 'prestaciones', 'jubilación', 'viudedad', 'pensiones', 'incapacidad', 'percepciones', 'asimiladas', 'requeridos', 'aeat', 'dicho', 'ejercicio', 'actualmente', 'dichos', 'campos', 'parecen', 'contemplados', 'tampoco', 'rama', 'módulo', 'provocando', 'fichero', 'generado', 'válido', 'importación', 'dado', 'campos', 'sido', 'incorporados', 'rama', 'consideramos', 'backport', 'cambios', 'hacia', 'permitiría', 'mantener', 'compatibilidad', 'modelo', '190', 'requis